In [1]:
import torch 
from torch import nn,optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm
import re 
import collections
import itertools
import MeCab
import neologdn
import emoji
#Mecab　path 確認コマンド
#echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
mecab = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')  # バグ対処

'\n'

# 補助関数の作成

In [2]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")



unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
#     #不要な文字を削除
#     text = remove_marks_regex.sub("", text)
    #?!.と単語の間に空白を挿入
    text = shift_marks_regex.sub(r"\1", text)
    #重ね表現の削除
    text = neologdn.normalize(text)
    #url削除
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    #絵文字削除
    text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])
    #桁区切りの削除
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text)
    text = re.sub(r'\d+', '0', text)
    # 半角記号の置換
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text = re.sub(u'[■-♯]', ' ', text)
    return text

def parse_line(line):
    line = normalize(line.strip())
    #翻訳元と翻訳先それぞれのトークンリストを作成する
    src,trg = line.split("\t")[:2]
    src = mecab.parse(src)
    trg = mecab.parse(trg)
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens

def build_vocab(tokens):
    #ファイル中のすべての文章でのトークン数を数える
    counts = collections.Counter(tokens)
    #トークンの出現数の多い順に並べる
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    #3つのタグを追加して正引きリストと逆引き用辞書を作る
    word_list = ["<UNK>", "<SOS>", "<EOS>"] \
    + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    #末尾に終了タグをつける
    words = words + ["<EOS>"]
    #辞書を利用して数値のリストに変換する
    words = [word_dict.get(w,0) for w in words]
    seq_len = len(words)
    #長さがmax_len以下の場合はパディングする
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    #Tensorに変換して返す
    return torch.tensor(words, dtype=torch.int64), seq_len

# TranslationPairDatasetクラスの作成

In [3]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        #単語数が多い文書をフィルタリングする関数
        def filter_pair(p):
            return not(len(p[0]) > max_len or len(p[1]) > max_len)
        #ファイルを開き、パース/フィルタリングする
        with open(path)as fp:
            pairs = map(parse_line, fp)
#             pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        #文書のペアをソースとターゲットに分ける
        src = [p[0][0:250] for p in pairs]
        trg = [p[1][0:50] for p in pairs]
        #それぞれの語彙集を作成する
        self.src_word_list, self.src_word_dict = build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = build_vocab(itertools.chain.from_iterable(trg))
        #語彙集を使用してTensorに変換する
        self.src_data = [words2tensor(words[:249], self.src_word_dict, max_len) for words in src]
        self.trg_data = [words2tensor(words[:49], self.trg_word_dict, 50, -100) for words in trg]
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src, Isrc = self.src_data[idx]
        trg, Itrg = self.trg_data[idx]
        return src, Isrc, trg, Itrg

# Encoderの作成

In [4]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=10, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
    def forward(self, x, h0=None,l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h

# Decoderの作成

In [5]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x =nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h

In [ ]:
class LongAttention:
    def __init__(self, units=300):
        self.dot = torch.dot(axes=[2, 2])
        self.attention = Activation(activation='softmax', name='attn_out')
        self.context = torch.dot(axes=[2,1])
        self.concat = torch.cat
        

# Translate

In [6]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    #Encoderで使用するので入力の長さもリストにしておく
    seq_len = [seq_len]
    #開始トークン準備
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    #入力文字配列をEncoderに入れてコンテキストを得る
    ctx = enc(input_tensor, l=seq_len)
    #開始トークンとコンテキストをDecoderの初期値にセット
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        #Decoderで次の単語を予測
        o,h = dec(z.view(1,1), h)
        #線形層の出力が最も大きい場所が次の単語のID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        #次の入力は今回の出力のIDを使用する
        z = wi
    #記録しておいた出力のIDを文字列に変換
    return " ".join(ds.trg_word_list[i] for i in results)

In [7]:
batch_size = 32
max_len = 500
path = "train_data/Text.tsv"
ds = TranslationPairDataset(path, max_len=max_len)
loder = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)


## 単語数確認

In [8]:
print(len(ds.src_word_list))
print(len(ds.trg_word_list))

61948
16387


In [9]:
# サンプルテキスト
text1 = "もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？　さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。\
「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」さてそんなとき、独女はどう対応したらいいか？ \
最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。\
そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、\
その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。\
一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）\
「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」"
text2 = "　今回配信されるのは、ドラマ『モテキ』でCMに切り替わる際、主人公・森山未來が神輿に担がれるシーンにて使用されていた、「好きよ！抱いて！」のフレーズでお馴染みの映画バージョンの着信ボイス。\
その他にもレコチョクでは、映画『モテキ』のオープニングテーマ曲のフジファブリック「夜明けのBEAT」や、メインテーマ曲の女王蜂「デスコ」など関連曲の着うた(R)、着うたフル(R)などを配信中。後日、\
出演者の長澤まさみ動画コメント配信や、ポラロイド写真のプレゼントも実施予定となっている。\
モテキ関連楽曲 映画「モテキ」オープニングテーマ：フジファブリック「夜明けのBEAT」映画「モテキ」メインテーマ：女王蜂「デスコ」 \
Half-Life「J-POP」大江千里「格好悪いふられ方」TM NETWORK 「SELF CONTROL」N'夙川BOYS「物語はちと?不安定」Fishmans 「いかれたBaby」Perfume「Baby cruising Love」 \
ももいろクローバー「走れ!」三木道三 「Lifetime　Respect」ACO 「悦びに咲く花」橘いずみ「失格」加藤 ミリヤ×清水 翔太「Love Forever」JUDY AND MARY 「LOVER SOUL」くるり「東京」・モテキ - 公開情報・ドコモマーケット MUSICストア"

## 動作確認

In [10]:
enc = Encoder(len(ds.src_word_list))
dec = Decoder(len(ds.trg_word_list))
translate("なぜ 来る の", enc, dec)

'もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも もっとも'

## モデルの学習

In [11]:
enc = Encoder(len(ds.src_word_list))
dec = Decoder(len(ds.trg_word_list))
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.001)
opt_dec = optim.Adam(dec.parameters(), 0.001)
loss_f =  nn.CrossEntropyLoss()

# BLEU

In [12]:
from collections import defaultdict
from nltk import bleu_score
def evalute_bleu(X, y):
    X = mecab.parse(X)
    y = mecab.parse(y)
    return bleu_score.sentence_bleu(X, y)
evalute_bleu(text1, translate(text1, enc, dec, max_len=max_len, device="cuda:0"))

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

4.247878894555134e-232

In [13]:
import pandas as pd
test = pd.read_csv("train_data/Text.tsv",  delimiter='\t', header=None)
for i in test.sample(n=3).values:
    print(i[1])

ディズニーの総力を結集した夢の映像プロジェクトがパッケージ化
お父さんや兄弟が理想の男性？
29日はアキバへGo！　レビューアー100人以上を大募集の太っ腹企画開催！


## モデルの学習部分作成

In [ ]:
from statistics import mean

def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)

for epoch in range(200):
    enc.train(), dec.train()
    losses = []
    bleues = []
    for x, lx, y, ly in tqdm.tqdm(loder):
        
        #xのPackedSequenceを作るために翻訳元の長さで降順にソート
        lx, sort_idx = lx.sort(descending = True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        #翻訳元をEncoderに入れてコンテキストを得る
        ctx = enc(x, l=lx)
    
        #yのpackedSequenceを作るために翻訳先の長さで降順にソート
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        #Decoderの初期値をセット
        h0 = (ctx[0][:,sort_idx,:],ctx[1][:,sort_idx,:])
        z = y[:,:-1].detach()
        #-100のままだとEmbeddingの計算でエラーが出るので値を0にしておく
        z[z==-100] = 0
        #Decoderに通して損失関数を計算
        o, _ = dec(z, h0,l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:,1:max(ly)]).squeeze())
        #誤差逆伝播
        enc.zero_grad()
        dec.zero_grad()
        loss.backward()
        opt_enc.step()
        opt_dec.step()
        losses.append(loss.item())
        
    
    
    
    enc.eval()
    dec.eval()
    print(epoch, mean(losses))
    for i in test.sample(n=50).values:
        bleues.append(evalute_bleu(i[0], translate(i[0], enc, dec, max_len=max_len, device="cuda:0")))
    print("BLEU SCORE:{}".format(mean(bleues)))
    with torch.no_grad():
        for i in test.sample(n=3).values:
            print(translate(i[0], enc, dec, max_len, device="cuda:0"))

torch.save(enc.state_dict(), './enc.pth')
torch.save(dec.state_dict(), './dec.pth')

100%|██████████| 231/231 [01:23<00:00,  2.76it/s]
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram ord

0 3.7680259339221114


  0%|          | 0/231 [00:00<?, ?it/s]

BLEU SCORE:1.587262925779543e-231
sports
sports
sports


 27%|██▋       | 63/231 [00:22<01:02,  2.71it/s]

In [12]:
import pandas as pd
tsv = pd.read_csv('tmup.tsv', delimiter='\t')
with open("tmp.txt", "w") as f:
    for i in tsv[["sentence_A_ja", "sentence_B_ja"]].values:
        f.write(i[0] + "\t" + i[1] + "\n")
#         f.write(i[0].replace(" ", "") + "\t" + i[1].replace(" ", "") + "\n")